### Starbucks Capstone Challenge
# System Demonstration

This notebook aims to demonstrate how the neural network trained in project would be part of a direct marketing system.

### Import the necessary libraries

In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import sklearn.metrics as metrics
from sklearn.preprocessing import scale

from models import RecurrentNN

In [2]:
pd.set_option('display.precision', 2)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

### Load the datasets

In [3]:
!unzip -o dataloaders.zip
train_dataloader, valid_dataloader, test_dataloader = \
    torch.load('dataloaders.pt')

Archive:  dataloaders.zip
  inflating: dataloaders.pt          


### Load the trained model

In [4]:
predictor = RecurrentNN(input_size=16, output_size=2,
                        hidden_size=128, hidden_layers=2)

## Load the saved model
predictor.load_state_dict(
    torch.load('recurrent_classifier.pt'))
predictor.eval()

print('Model loaded successfully')

Model loaded successfully


### Load and transform the portfolio

In [5]:
## Load the portfolio data and transform it to feed the networks
data_dir = './data'
portfolio_data_path = os.path.join(data_dir, 'portfolio.json')
portfolio_df = pd.read_json(portfolio_data_path, orient='records', lines=True)

# Set id as index
portfolio_df.set_index(keys='id', verify_integrity=True, inplace=True)

display(portfolio_df.style.set_caption('Portfolio (raw values)'))


# Make offer_type one hot encoded
portfolio_df = portfolio_df.join(
    pd.get_dummies(portfolio_df.pop('offer_type')))

# Transform channels in distinct features
channels_df = pd.DataFrame(portfolio_df.pop('channels'))
channels_df = channels_df.explode('channels')
channels_df = channels_df.assign(value=lambda x: 1)
channels_df = channels_df.pivot(columns='channels', values='value')
channels_df.fillna(value=0, inplace=True)
portfolio_df = portfolio_df.join(channels_df)
channels_df = None

# Remove email column
portfolio_df.drop(columns='email', inplace=True)

# Scale values
data = portfolio_df[['reward','difficulty','duration']]
data = scale(data)
portfolio_df[['reward','difficulty','duration']] = data

# Include the case of not sending an offer
portfolio_df = portfolio_df.append(pd.DataFrame(
    [[0, 0, 0, 0, 0, 0, 0, 0, 0]],
    columns=portfolio_df.columns,
    index=['no_offer_sending']))

display(portfolio_df.style.set_caption('Portfolio (transformed)'))

,reward,channels,difficulty,duration,offer_type
id,,,,,
ae264e3637204a6fb9bb56bc8210ddfd,10,"['email', 'mobile', 'social']",10,7,bogo
4d5c57ea9a6940dd891ad53e9dbe8da0,10,"['web', 'email', 'mobile', 'social']",10,5,bogo
3f207df678b143eea3cee63160fa8bed,0,"['web', 'email', 'mobile']",0,4,informational
9b98b8c7a33c4b65b9aebfe6a799e6d9,5,"['web', 'email', 'mobile']",5,7,bogo
0b1e1539f2cc45b7b9fa7c272da2e1d7,5,"['web', 'email']",20,10,discount
2298d6c36e964ae4a3e7e9706d1fb8c2,3,"['web', 'email', 'mobile', 'social']",7,7,discount
fafdcd668e3743c1bb461111dcafc2a4,2,"['web', 'email', 'mobile', 'social']",10,10,discount
5a8bc65990b245e5a138643cd4eb9837,0,"['email', 'mobile', 'social']",0,3,informational
f19421c1d4aa40978ebb69ca19b0e20d,5,"['web', 'email', 'mobile', 'social']",5,5,bogo


,reward,difficulty,duration,bogo,discount,informational,mobile,social,web
ae264e3637204a6fb9bb56bc8210ddfd,1.71,0.42,0.23,1,0,0,1.00,1.00,0.00
4d5c57ea9a6940dd891ad53e9dbe8da0,1.71,0.42,-0.68,1,0,0,1.00,1.00,1.00
3f207df678b143eea3cee63160fa8bed,-1.24,-1.39,-1.14,0,0,1,1.00,0.00,1.00
9b98b8c7a33c4b65b9aebfe6a799e6d9,0.24,-0.49,0.23,1,0,0,1.00,0.00,1.00
0b1e1539f2cc45b7b9fa7c272da2e1d7,0.24,2.22,1.59,0,1,0,0.00,0.00,1.00
2298d6c36e964ae4a3e7e9706d1fb8c2,-0.35,-0.13,0.23,0,1,0,1.00,1.00,1.00
fafdcd668e3743c1bb461111dcafc2a4,-0.65,0.42,1.59,0,1,0,1.00,1.00,1.00
5a8bc65990b245e5a138643cd4eb9837,-1.24,-1.39,-1.59,0,0,1,1.00,1.00,0.00
f19421c1d4aa40978ebb69ca19b0e20d,0.24,-0.49,-0.68,1,0,0,1.00,1.00,1.00
2906b810c7d4411798c6938adc9daaa5,-0.65,0.42,0.23,0,1,0,1.00,0.00,1.00


# Demonstrate the system in use

## Analyze one specific customer
In this case, we analyze one specific customer to understand how past events affect the choice for a new offer sending.

Here, we pick one customer in the test dataset, so the features are already transformed. Otherwise, it is necessary to transform the data in the same manner made in the Feature Engineering phase.

Each offer from the portfolio dataset is concatenated to the customer data so that we have a feature vector.

The customer history is given by a sequence of feature vectors according to the offer received in past moments.

To obtain the probabilities of the offer acceptance, first, we concatenate the customer history until the moment and the new feature vector. Then, we pass into the network the new sequence generated by that concatenation.

In this example, I split the customer history in timesteps to obtain which offer would be the most appropriate in each past moment.

In [6]:
with torch.no_grad():
    # Get one batch of customer histories
    cust_history_list, cust_y_true_list = next(iter(test_dataloader))
    # Get the history of one customer
    cust_history = cust_history_list[8],
    cust_y_true = cust_y_true_list[8]
    
    print("Offers sent by the current system")
    print(cust_history[0].squeeze()[:,:9].numpy())
    
    print("True labels for the offers sent")
    print(cust_y_true.numpy())
    
    # Get the customer demographic data
    cust_data = cust_history[0][0][-7:]

    # Get the data of each offer
    offer_data = portfolio_df.reset_index().values
    
    # Build features by concatenating offer data + customer data
    # Create one feature vector for each offer
    features = np.repeat(cust_data.view(1,-1), 11, axis=0)
    features = np.concatenate((offer_data[:,1:], features), axis=1)
    features = torch.tensor(features.tolist())
    features = features.unsqueeze(1) # batch: 11 offers; seq_length: 1 history

    # Calculate the probabilities after each interaction
    y_pred_moments = []
    for moment in range(7):
        history = cust_history[0][:moment].unsqueeze(0)
        history = history.repeat(11,1,1)
        features_moment = torch.cat((history, features), dim=1)
        y_pred = predictor(features_moment)
        y_pred = torch.softmax(y_pred, dim=2)
        y_pred_moments.append(y_pred[:,-1,1].numpy()*100)
#         if moment == 6:
#             print(cust_history)
#             print(cust_y_true)
#             print(y_pred[-1].topk(1,dim=1)[1].squeeze())
# #             print(y_pred[-1])


y_pred_df = pd.DataFrame(y_pred_moments,
                         columns=portfolio_df.index,
                         index=['moment_1', 'moment_2', 'moment_3',
                                'moment_4', 'moment_5', 'moment_6',
                                'moment_7']).T
y_pred_df = y_pred_df.style.background_gradient(cmap='YlGn', axis=0)
y_pred_df.set_caption('Probabilites at each moment')
display(y_pred_df)

Offers sent by the current system
[[-1.2352941  -1.3917431  -1.1351916   0.          0.          1.
   1.          0.          1.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 1.7058823   0.41571546 -0.6811149   1.          0.          0.
   1.          1.          1.        ]
 [ 0.23529412 -0.4880138   0.22703831  1.          0.          0.
   1.          0.          1.        ]]
True labels for the offers sent
[1 1 0 1 0 0]


,moment_1,moment_2,moment_3,moment_4,moment_5,moment_6,moment_7
ae264e3637204a6fb9bb56bc8210ddfd,21.19,20.31,23.07,26.90,26.05,26.70,24.51
4d5c57ea9a6940dd891ad53e9dbe8da0,19.78,18.13,24.54,26.08,26.62,21.00,21.45
3f207df678b143eea3cee63160fa8bed,28.20,31.95,34.33,34.22,35.31,34.30,33.74
9b98b8c7a33c4b65b9aebfe6a799e6d9,18.88,20.04,19.80,22.78,21.50,21.73,19.26
0b1e1539f2cc45b7b9fa7c272da2e1d7,11.58,10.73,10.46,12.32,11.09,11.16,8.48
2298d6c36e964ae4a3e7e9706d1fb8c2,57.81,58.23,65.59,64.70,65.09,60.91,61.09
fafdcd668e3743c1bb461111dcafc2a4,64.11,62.81,70.18,68.56,68.51,63.59,63.52
5a8bc65990b245e5a138643cd4eb9837,50.08,55.24,58.99,58.74,60.61,63.74,63.68
f19421c1d4aa40978ebb69ca19b0e20d,39.28,40.39,46.60,48.14,48.43,45.16,45.48
2906b810c7d4411798c6938adc9daaa5,22.32,22.81,25.03,25.87,25.25,22.47,20.61


## Make predictions for a batch of customers
In this case, one batch of customers' history is passed into the network to get predictions for the next offer sending moment.

In the resulting table, each column represents one customer. Each row, one offer. Each cell, the probability of completion for that offer given that customer.

In [7]:
with torch.no_grad():
    ## Get one batch of customer history
    cust_history_list, cust_y_true_list = next(iter(test_dataloader))

    ## For each customer, make predictions for each offer
    cust_result = []
    for cust_history, cust_y_true in zip(cust_history_list, cust_y_true_list):
        # Get the customer demographic data
        cust_data = cust_history[0][-7:].unsqueeze(0)

        # Get the data of each offer
        offer = portfolio_df.reset_index().values

        # Build features by concatenating offer data + customer data
        # Create one feature vector for each offer
        cust_data = np.repeat(cust_data, 11, axis=0)
        feat_offer = np.concatenate((offer[:,1:], cust_data), axis=1)
        feat_offer = torch.tensor(feat_offer.tolist())

        # Concatenate the offers after the user history
        # Once for each offer
        feat_offer = feat_offer.unsqueeze(1)
        cust_history = cust_history.repeat(11,1,1)
        features = torch.cat((cust_history,feat_offer),dim=1)

        # Calculate the probabilities after each offer
        y_linear = predictor(features)
        y_linear = torch.softmax(y_linear, dim=2)

        # Store the probabilities for this customer
        cust_result.append(y_linear[:,-1,1].numpy()*100)

cust_df = pd.DataFrame(cust_result, columns=portfolio_df.index).T
cust_df = cust_df.style.background_gradient(cmap='YlGn')
display(cust_df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
ae264e3637204a6fb9bb56bc8210ddfd,37.11,40.32,41.18,32.69,2.47,41.95,37.59,23.12,24.51,46.21,36.26,35.90,3.45,17.06,47.38,43.60,39.75,3.53,21.11,46.63,26.50,2.91,31.70,13.31,49.75,2.88,37.20,41.64,36.45,47.25,42.42,37.25
4d5c57ea9a6940dd891ad53e9dbe8da0,33.69,48.60,33.41,31.28,2.21,46.96,43.69,20.52,21.45,41.23,41.77,34.63,2.39,15.67,59.31,56.06,46.26,2.41,18.55,52.61,23.40,2.48,32.19,12.73,52.86,2.60,35.93,50.65,44.20,56.48,48.08,43.73
3f207df678b143eea3cee63160fa8bed,42.43,23.50,45.66,31.48,18.93,24.76,20.26,24.07,33.74,40.27,20.99,42.44,20.83,18.32,32.18,38.73,18.36,19.64,22.99,35.13,31.33,30.68,20.05,24.51,39.46,20.31,24.12,25.04,19.05,43.69,26.28,21.79
9b98b8c7a33c4b65b9aebfe6a799e6d9,29.10,32.90,29.53,27.90,4.99,33.11,29.00,19.54,19.26,29.94,27.80,24.79,6.19,14.76,39.23,41.96,27.43,6.79,16.89,37.96,21.55,7.39,23.51,13.03,37.33,5.87,26.32,32.04,26.81,42.17,33.00,27.50
0b1e1539f2cc45b7b9fa7c272da2e1d7,13.38,20.99,14.96,16.61,1.43,21.56,17.47,11.53,8.48,14.96,16.97,10.71,1.66,8.72,22.48,24.11,18.79,1.97,8.01,22.63,10.45,1.62,15.55,5.50,20.63,1.76,17.33,19.86,17.39,21.66,20.37,17.77
2298d6c36e964ae4a3e7e9706d1fb8c2,72.80,62.52,74.49,55.39,14.77,57.64,50.03,39.38,61.09,74.81,51.56,73.88,18.70,31.16,74.09,80.05,52.29,12.15,40.48,73.64,57.91,31.65,43.08,39.58,77.22,16.61,47.43,59.32,48.67,79.39,59.82,51.71
fafdcd668e3743c1bb461111dcafc2a4,75.40,65.83,76.84,54.27,12.98,57.21,48.69,38.69,63.52,76.21,51.74,74.90,17.77,30.25,76.01,82.50,53.64,9.52,39.19,75.18,60.08,32.88,42.67,38.97,78.81,14.97,45.73,59.76,49.53,80.30,59.32,52.13
5a8bc65990b245e5a138643cd4eb9837,70.49,30.88,77.67,49.23,34.07,34.16,27.56,40.04,63.68,72.36,29.32,74.46,40.79,30.79,42.10,48.78,25.64,34.71,42.76,50.43,56.37,52.24,30.44,45.20,61.12,34.58,38.68,33.16,24.79,58.93,37.00,29.98
f19421c1d4aa40978ebb69ca19b0e20d,59.16,52.90,60.39,46.23,9.36,50.85,45.41,32.64,45.48,62.70,45.46,59.16,11.61,25.19,64.76,67.71,45.62,9.67,32.89,63.44,44.67,16.20,37.75,28.59,66.96,10.58,42.71,52.27,43.27,69.63,52.95,45.47
2906b810c7d4411798c6938adc9daaa5,30.01,29.78,29.92,26.34,5.34,28.00,23.16,18.73,20.61,28.98,24.24,26.63,5.97,14.27,37.23,43.45,23.79,5.63,15.69,36.45,21.84,8.40,20.82,14.02,36.72,6.32,23.09,29.02,23.54,41.63,28.70,25.09
